In [1]:
"""
Created on Thu Jan 15 11:00:00 2020
Group 4
Sub Group : Sentiment
@author : FR , CBO, JC, NM
PreProcessing  
"""

'\nCreated on Thu Jan 15 11:00:00 2020\nGroup 4\nSub Group : Sentiment\n@author : FR , CBO, JC, NM\nPreProcessing  \n'

In [2]:
# Auto PEP8
#%load_ext pycodestyle_magic
#%pycodestyle_on

In [3]:
import pandas as pd
import string
import numpy as np
import sklearn
import spacy
import en_core_web_sm
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import ne_chunk
from nltk import pos_tag
from nltk import ne_chunk
from nltk import pos_tag
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import remove_stopwords
import scipy
from langdetect import detect
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\socro\AppData\Roaming\Python\Python36\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\socro\AppData\Roaming\Python\Python36\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\socro\AppData\Roaming\Python\Python36\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\socro\Anaconda3\lib\imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\socro\Anaconda3\lib\imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Le module spécifié est introuvable.

During handling of the above exception, another

TypeError: must be str, not list

In [ ]:
data = pd.read_excel('../data/ALL_DATA_ANNOTATE.xlsx')

In [ ]:
review = data['Sentence']

In [ ]:
label=data
label.drop(['Data_Source', 'Airline_Name', 'Aircraft_Type', 'Cabin_Class',
       'Type_Of_Lounge', 'Type_Of_Traveller', 'Date_Visit', 'Date_Flown',
       'Airport', 'Route', 'Cabin_Staff_Service', 'Lounge_Staff_Service',
       'Bar_And_Beverages', 'Food_And_Beverages', 'Ground_Service', 'Catering',
       'Cleanliness', 'Comfort', 'Aisle_Space', 'Wifi_And_Connectivity',
       'Inflight_Entertainment', 'Viewing_Tv_Screen', 'Power_Supply', 'Seat',
       'Seat_Type', 'Seat_Comfort', 'Seat_Legroom', 'Seat_Storage',
       'Seat_Width', 'Seat_Recline', 'Washrooms', 'Value_For_Money',
       'Overall_Customer_Rating', 'Recommended', 'Date_Review', 'Review','Unnamed: 54','Sentence'], axis=1, inplace = True)

### Pre processing

In [8]:
def clean_wo_divide(commentary: string):
    """
    Documentation
        This functon drop the title of comment 
    Parameters:
        commentary: a commentary 
    Output:
        result: the commentary without 'Points positifs' or 'Points négatifs'
    """
    commentary = commentary.replace("Points positifs :", " ")
    commentary = commentary.replace("Points négatifs :", " ")
    return commentary

In [9]:
def label_mean(labels):
    """
    Documentation
    Parameters:
        commentary: labels
    Output:
        result: labels clean
    """
    labels = labels.replace(2, 1)
    labels = labels.mean(axis = 1)
    labels = labels.fillna(2) # 2 is neutral, we will replace it by 0 later to avoid them to be considered as negative
    labels = labels.round()
    labels = labels.replace(0, -1) # 0 (negative) is now -1 (easier to understand)
    labels = labels.replace(2, 0) # Neutral is 0
    return labels

In [10]:
def english_only(df: string, label: int):
    """
    Documentation
        This function keep only comment in english
    Parameters:
        data: a commentary and label
    Output:
        result: Comment english and label
    """
    idx_to_drop = []
    for idx, com in enumerate(df):
        if detect(com) != 'en':
            idx_to_drop.append(idx)
    df = df.drop(df.index[idx_to_drop])
    label = label.drop(label.index[idx_to_drop])
    label = label.reset_index(drop=True)
    label.shape
    return df, label

In [11]:
def lemmatize(sentence: string):
    """
    Documentation
    Parameters:
        sentence: a sentence from a commentary
    Output:
        lemmatized: the lemmatized sentence
    """
    nlp = spacy.load('en_core_web_sm')
    s = nlp(str(sentence))
    lemmatized = ''
    for w in s:
        lemmatized += w.lemma_ + ' '
    return lemmatized

In [12]:
def clean(commentary: string):
    """
    Documentation:
        Clean the comment
    Parameters:
        commentary: a commentary from 'OTHER_DATA_ANNOTATE.xlsx'
    Output:
        result: the cleaned commentary
    """
    # Convert text to lowercase
    commentary = str(commentary).lower()
    # Remove punctuation
    commentary = commentary.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    stop_words.add('pron')
    tokens = word_tokenize(commentary)
    result = [i for i in tokens if not i in stop_words]
    result = " ".join(result)
    # Remove whitespaces
    result = result.strip()
    return result

In [152]:
def tf_idf(X_train: string, X_test: string):
    """
    Documentation:
    Parameters:
        X_train: Training DataFrame
        X_test: Testing DataFrame
    Output:
        result: tf idf on X_train and tf idf on X_test on the vocabulary of X_train
    """
    vectorizer = TfidfVectorizer()
    if X_train is not None:
        vectorizer.fit(X_train)
        X = vectorizer.transform(X_test)
        tf_idf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names(), index=X_test.index)
    else:
        X = vectorizer.fit_transform(X_train)
        tf_idf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names(), index=X_train.index)
    print('Taille de la matrice X : ', X.shape)
    return tf_idf

In [14]:
def main(commentary: string, y: int):
    """
    Documentation:
    Parameters:
        commentary: data
        y: label of data
    Output:
        commentary: data clean
        y: label clean
    """
    commentary_clean, label = english_only(commentary, y)
    #commentary_clean = commentary_clean.apply(lemmatize)
    commentary_clean = commentary_clean.apply(clean_wo_divide)
    commentary_clean = commentary_clean.apply(clean)
    return (commentary_clean, label)

In [ ]:
def preprocessing_neural_network(X_train, X_test, X_pred, max_features=10000, max_words=120, num_classes=3):
    """
    Documentation:
    Parameters:
        X_train: Training DataFrame
        X_test: Testing DataFrame
        max_features: the number of rows to take in input
        max_words: the maximum amout of words that will be encoded for one row
        num_classes: number of labels to predict
    Output:
        X_train: Training DataFrame
        X_test: Testing DataFrame
    """
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(X_train)
    X_train = tokenizer.texts_to_sequences(list(X_train))
    X_test = tokenizer.texts_to_sequences(list(X_test))
    X_pred = tokenizer.texts_to_sequences(list(X_pred))
    # Pad the sequences to have input with same length (max_words)
    X_train = pad_sequences(X_train, maxlen=max_words)
    X_test = pad_sequences(X_test, maxlen=max_words)
    X_pred = pad_sequences(X_pred, maxlen=max_words)
    return (X_train, X_test, X_pred)

We choose to remove non-english commentaries due to their low presence.    
Then we choose to split the dataframe into a train-test datasets in order to see if the model isn't overfitting. We will then crossvalidate the best model.    

We apply tf_idf on the list of cleaned sentences to have the most relevant words in our tf-idf matrix.

In [85]:
#data, label = main(review, label_mean(label))

In [220]:
#X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.30, random_state=42)

In [221]:
"""tf_idf_train = tf_idf(X_train, X_train)
tf_idf_test = tf_idf(X_train, X_test)"""

Taille de la matrice X :  (3966, 4763)
Taille de la matrice X :  (1700, 4763)


Preprocessing of the input of the neural network.
We remarked that we have better scores if we put raw data as input, that's why we choose to only tokenize the sentences.    
Then we pad the sentences with zeros (that will be ignored by the model) because the inputs must have the same length and since we don't want to loose information, we need to pad the sentence in accordance with the longest sentence.

In [223]:
"""target = label.values
# Get_dummies on labels to have 3 classes
y_neural_network = to_categorical(target, num_classes=num_classes)"""

In [224]:
#X_train_neural_network, X_test_neural_network, y_train_neural_network, y_test_neural_network = train_test_split(data, y_neural_network, test_size=0.30, random_state=42)

In [225]:
#X_train_neural_network, X_test_neural_network = preprocessing_neural_network(X_train, X_test,max_features=10000, max_words=120, num_classes=3)

In [155]:
def export(data, name: string):
    """
    Documentation:
    Parameters:
        data: export data 
        name: name of file
    Output:
        result: data export 
    """
    pd.DataFrame(data).to_csv("../data/"+name+".csv", sep=",")
    print('Matrice de ' + name + ' enregistrée')

In [229]:
#export(y_test_neural_network,"y_test_neural_network")

Matrice de y_test_neural_network enregistrée
